# Data Visualization and Statistics
Examines the raw training and test data, as well as new data pulled from external sources

In [ ]:
# All imports
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import utils

# Raw training and test data

In [ ]:
# Load raw data
raw_train_df = pd.read_csv(utils.RAW_TRAIN_PATH, dtype={"accession": str, "score": float})
raw_test_df = pd.read_csv(utils.RAW_TEST_PATH, dtype={"accession": str, "score": float})

In [ ]:
# TODO: visualize something about the data here